<a href="https://colab.research.google.com/github/whairan/ML_Notebooks/blob/main/Demo_Model_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ETL
# Alteryx, H20.ai, Spark

In [ ]:
# sklearn --> Pipeline


# Given --> Data in a structured (tabular) format!

# Imputation
# nominal_categorical_features = [] --> One Hot Encoding
# ordinal_categorical_features = [] --> OrdinalEncoding
# continous_features = [] --> Feature Scaling
# Model Building --> This is the best model that we want to deploy


from sklearn.datasets import make_classification
X, y = make_classification(n_samples = 200, n_features = 5, n_informative = 3, n_redundant = 2,  n_classes = 2, random_state = 1)

print(X.shape)
print(y.shape)

(200, 5)
(200,)


In [ ]:
import numpy as np
np.unique(y, return_counts = True)

(array([0, 1]), array([101,  99]))

In [ ]:
np.isnan(X).sum(axis = 0)

array([0, 0, 0, 0, 0])

In [ ]:
X[0,0] = np.nan

In [ ]:
np.isnan(X).sum(axis = 0)

array([1, 0, 0, 0, 0])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
#make_scorer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
# ColumnTransformer

our_steps = [('imp', SimpleImputer()), ('ss', StandardScaler()), ('dt', DecisionTreeClassifier())]

hyperparams = {'imp__strategy':['mean', 'median'],
               'dt__max_depth':range(3, 11),
               'dt__min_samples_split':range(2,11),
               'dt__min_samples_leaf':range(1, 5)}

our_pipeline = Pipeline(steps = our_steps)

from sklearn.model_selection import GridSearchCV, StratifiedKFold
skf = StratifiedKFold(n_splits = 5)
gcv = GridSearchCV(our_pipeline, param_grid = hyperparams, cv = skf, scoring = 'recall')
gcv.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('imp', SimpleImputer()),
                                       ('ss', StandardScaler()),
                                       ('dt', DecisionTreeClassifier())]),
             param_grid={'dt__max_depth': range(3, 11),
                         'dt__min_samples_leaf': range(1, 5),
                         'dt__min_samples_split': range(2, 11),
                         'imp__strategy': ['mean', 'median']},
             scoring='recall')

In [ ]:
gcv.best_params_

{'dt__max_depth': 4,
 'dt__min_samples_leaf': 2,
 'dt__min_samples_split': 3,
 'imp__strategy': 'mean'}

In [ ]:
gcv.best_estimator_

Pipeline(steps=[('imp', SimpleImputer()), ('ss', StandardScaler()),
                ('dt',
                 DecisionTreeClassifier(max_depth=4, min_samples_leaf=2,
                                        min_samples_split=3))])

In [ ]:
gcv.best_estimator_.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0])

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, gcv.best_estimator_['dt'].predict(X_test), pos_label = 0)

0.9545454545454546

In [ ]:
# Node --> How to parititon the data? --> test every feature

